<a href="https://colab.research.google.com/github/Mutai-Gilbert/data_extraction_neo4j/blob/main/data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j pandas openpyxl

from neo4j import GraphDatabase
import pandas as pd
import os
from datetime import datetime

class Neo4jConnection:

    def __init__(self, uri, user, password):
        self.__uri = uri
        self.__user = user
        self.__password = password
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    def test_connection(self):
        test_query = "MATCH (n) RETURN n LIMIT 5"
        session = None
        try:
            session = self.__driver.session()
            result = list(session.run(test_query))
            if result:
                print("Connection test successful. Nodes found:", len(result))
            else:
                print("Connection test completed but no nodes were found.")
        except Exception as e:
            print("Connection test failed:", e)
        finally:
            if session is not None:
                session.close()

    def query(self, query, parameters=None, db=None):
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

# uri = "bolt://dev1.rebl.ai:7687"
# user = "neo4j"
# password = "reblD3v1andtherestisjustfiller"  # Be cautious with passwords in code

uri = ""  # Adjust the port if needed; 7687 is the default for Neo4j.
user = "neo4j"
password = ""

conn = Neo4jConnection(uri=uri, user=user, password=password)

# Perform a connection test
conn.test_connection()

neo4j_query = """
MATCH (user:User)-[r]->(post:Post)
WHERE type(r) IN ['POSTED_ON_2024_04_07', 'POSTED_ON_2024_04_06', 'POSTED_ON_2024_04_05', 'POSTED_ON_2024_04_04', 'POSTED_ON_2024_04_03', 'POSTED_ON_2024_04_02', 'POSTED_ON_2024_04_01']
WITH user, COUNT(post) AS postsCount
ORDER BY postsCount DESC
RETURN user.uname AS UserName, postsCount
LIMIT 10
"""

# Execute the adjusted query
results = conn.query(neo4j_query)

# Check if results are empty
if not results:
    print("No data found for the last 7 days.")
else:
    # Convert results to DataFrame
    df = pd.DataFrame([dict(record) for record in results])

    # Create a directory with the current date
    current_date = datetime.now().strftime('%Y-%m-%d')
    dir_path = f'/content/{current_date}'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    # Save DataFrame to Excel
    excel_path = f'{dir_path}/user_post_{current_date}.xlsx'
    df.to_excel(excel_path, index=False)

    print(f'Excel file saved at: {excel_path}')
